In [1]:
import torch
import numpy as np
import cv2
from PIL import Image

In [2]:
import sys
sys.path.append('..')
from models import resnet50, densenet_121, resnet50_adv, resnet18, resnet18_seg
from utils import *

In [3]:
checkpoint_path = '../checkpoints/my_data_v6_resnet18/03_05_23_25_fold_0_model_best.pth.tar'
model = resnet18()
checkpoint = torch.load(checkpoint_path)
model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [4]:
cloud_dir = "G:\\vscode_workspace\\Weather_Recognition\\data_split_v6\\train\\Cloud"
snow_dir = "G:\\vscode_workspace\\Weather_Recognition\\data_split_v6\\snow_svm\\Snow"

In [5]:
from dataset import MyDataSegSnow, MyDataSegSnowSVM, MyData_S_C_SVM
from torchvision import transforms

In [6]:
img_size = (576, 720)
transform = transforms.Compose([
    # transforms.Resize((image_size, image_size)),
    transforms.Resize(img_size),
    transforms.ToTensor(),
    # transforms.Normalize(mean, std)
])

dataset = MyData_S_C_SVM(root_path="G:\\vscode_workspace\\Weather_Recognition\\data_split_v6\\train\\", transform=transform)

In [8]:
# 获取最后路面全连接层的特征
features_blobs = []
def hook_feature(module, input, output): # input是注册层的输入 output是注册层的输出
    # print("hook input",input[0].shape)
    # features_blobs.append(output)
    features_blobs.append(output.data.cpu().numpy())

In [10]:
features_blobs = []
# model.model_h.st_gcn_networks[2].sigmoid.register_forward_hook(hook_feature)
model.avgpool.register_forward_hook(hook_feature)
# model.eval()

In [13]:
features_blobs = []
features_labels = []
model = model.cuda()
model.eval()

data_iter = torch.utils.data.DataLoader(dataset, 4, shuffle=True)
for batch_idx, (inputs, targets) in enumerate(data_iter):
    inputs, targets = inputs.cuda(), targets.cuda()
    # inputs, targets = inputs.cuda(), targets.cuda()

    outputs = model(inputs)
    features_labels.append(targets.data.cpu().numpy())
    # if batch_idx == 1:
    #     break
    # outputs = model(inputs)
print(features_blobs[0].shape)

(4, 512, 1, 1)


In [14]:
features_x = np.concatenate(features_blobs, axis=0)
print(features_x.shape)
features_x = features_x.reshape(-1, 512)
print(features_x.shape)
features_y = np.concatenate(features_labels, axis=0)
print(features_y.shape)

(15570, 512, 1, 1)
(15570, 512)
(15570,)


In [15]:
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import accuracy_score

In [16]:
# 支持向量机预测
svc = SVC()
svc.fit(features_x, features_y)
# y_pred = svc.predict(x_test)
svc.score(features_x, features_y)
train_acc_svc = accuracy_score(svc.predict(features_x), features_y)
# test_acc_svc = accuracy_score(svc.predict(x_test), y_test)
print(train_acc_svc)
# svc.predict(x_test)

1.0


In [17]:
# 保存模型
import pickle
save_path = "sunny_cloud_svm.pkl"
with open(save_path, 'wb') as pfile:
    pickle.dump(svc, pfile)
# joblib.dump(svc, 'snow_svm.pkl')

In [19]:
with open("sunny_cloud_svm.pkl", "rb") as f:
    svc2 = pickle.load(f)
train_acc_svc = accuracy_score(svc2.predict(features_x), features_y)
# test_acc_svc = accuracy_score(svc.predict(x_test), y_test)
print(train_acc_svc)

1.0


In [20]:
# 在测试集中看准确率
dataset = MyData_S_C_SVM(root_path="G:\\vscode_workspace\\Weather_Recognition\\data_split_v6\\test\\", transform=transform)

In [21]:
features_blobs = []
features_labels = []
model = model.cuda()
model.eval()

data_iter = torch.utils.data.DataLoader(dataset, 4, shuffle=True)
for batch_idx, (inputs, targets) in enumerate(data_iter):
    inputs, targets = inputs.cuda(), targets.cuda()
    # inputs, targets = inputs.cuda(), targets.cuda()

    outputs = model(inputs)
    features_labels.append(targets.data.cpu().numpy())
    # if batch_idx == 1:
    #     break
    # outputs = model(inputs)
print(features_blobs[0].shape)

(4, 512, 1, 1)


In [22]:
test_features_x = np.concatenate(features_blobs, axis=0)
print(test_features_x.shape)
test_features_x = test_features_x.reshape(-1, 512)
print(test_features_x.shape)
test_features_y = np.concatenate(features_labels, axis=0)
print(test_features_y.shape)

(4470, 512, 1, 1)
(4470, 512)
(4470,)


In [23]:
train_acc_svc = accuracy_score(svc2.predict(test_features_x), test_features_y)
# test_acc_svc = accuracy_score(svc.predict(x_test), y_test)
print(train_acc_svc)

0.7465324384787472
